In [1]:
import os
try:                  # get to root of project
    print(od)
except NameError:
    od = os.getcwd()
    
os.chdir(od + '/../..')
print(os.getcwd())

/home/mike/py/mylib/CacheGrab


In [37]:
import collections
import requests
import json
from time import sleep

In [3]:
type(requests)

module

In [4]:
import cachegrab as cg

In [5]:
from cachegrab.utils import dummy

In [6]:
resp = dummy.get('foo', params=None)
print(type(resp))
resp.json()

<class 'requests.models.Response'>


{'RetrievedDate': '2017-12-13T07:45:09.395578',
 'done': True,
 'records': [{'AccountId': '1337BEEF',
   'Amount': 31415.926,
   'CreatedDate': '2015-07-13T07:30:24.000+0000',
   'IsClosed': True,
   'IsWon': False,
   'Name': 'FooBar Opportunity AAA',
   'attributes': {'type': 'Opportunity', 'url': '/services/data/foo/bar'}}],
 'totalSize': 1}

In [42]:
def try_json(*args, **kwargs):
    # Pack args and kwargs into a JSON. If this fails, it's because one or more of the arguments is not serializable
    print("Args: {}".format(args))
    print("kwArgs: {}".format(kwargs))
    data = {'args': args, 'kwargs': kwargs}
    j = json.dumps(data)
    return(j)

try_json('foo', 'bar', spam=2, eggs=True, pi=3.14)

Args: ('foo', 'bar')
kwArgs: {'eggs': True, 'pi': 3.14, 'spam': 2}


'{"args": ["foo", "bar"], "kwargs": {"eggs": true, "pi": 3.14, "spam": 2}}'

In [71]:
jtest = try_json('foo', 'bar', spam=2, eggs=True, pi=3.14)
jdict = json.loads(jtest)
print(jdict['args'])
print(jdict['kwargs'])

try_json(*jdict['args'], **jdict['kwargs'])

Args: ('foo', 'bar')
kwArgs: {'eggs': True, 'pi': 3.14, 'spam': 2}
['foo', 'bar']
{'eggs': True, 'pi': 3.14, 'spam': 2}
Args: ('foo', 'bar')
kwArgs: {'eggs': True, 'pi': 3.14, 'spam': 2}


'{"args": ["foo", "bar"], "kwargs": {"eggs": true, "pi": 3.14, "spam": 2}}'

In [7]:
class memoized(object):
    '''Basic Memoizing Decorator. Caches a function's return value each time it is called.
    If called later with the same arguments, the cached value is returned
    (not reevaluated).
    From the python website, https://wiki.python.org/moin/PythonDecoratorLibrary#Memoize
    '''

    def __init__(self, func):
        self.func = func
        self.cache = {}

    def __call__(self, *args):
        if not isinstance(args, collections.Hashable):
            # uncacheable. a list, for instance.
            # better to not cache than blow up.
            return self.func(*args)
        if args in self.cache:
            return self.cache[args]
        else:
            value = self.func(*args)
            self.cache[args] = value
            return value

    def __repr__(self):
        '''Return the function's docstring.'''
        return self.func.__doc__

    def __get__(self, obj, objtype):
        '''Support instance methods.'''
        return functools.partial(self.__call__, obj)


In [60]:
class hmemoized(object):
    '''Hashing Memoizing Decorator. Caches a function's return value each time it is called.
    If called later with the same arguments, the cached value is returned
    (not reevaluated).
    From the python website, https://wiki.python.org/moin/PythonDecoratorLibrary#Memoize
    '''

    def __init__(self, func):
        self.func = func
        print(self.func)
        self.cache = {}

    def __call__(self, *args, **kwargs):
        key = try_json(*args, **kwargs) # will raise a TypeError from JSON if any args or kwargs are not serializable
        if key in self.cache:
            return self.cache[key]
        else:
            value = self.func(*args, **kwargs)
            self.cache[key] = value
            return value

    def __repr__(self):
        '''Return the function's docstring.'''
        return self.func.__doc__

    def __get__(self, obj, objtype):
        '''Support instance methods.'''
        return functools.partial(self.__call__, obj)

In [61]:
dd = requests.Response()

In [62]:
dd._content = bytes('fooz', 'utf8') 

In [63]:
dd.text

'fooz'

In [64]:
x = 'fooz'

In [65]:
@memoized
def get(url, params=None, delay=1):
    """Some sort of docstring"""
    return dummy.get(url, params=params, delay=delay)

In [66]:
get('foo', None, 1).json()

{'RetrievedDate': '2017-12-13T09:06:44.322059',
 'done': True,
 'records': [{'AccountId': '1337BEEF',
   'Amount': 31415.926,
   'CreatedDate': '2015-07-13T07:30:24.000+0000',
   'IsClosed': True,
   'IsWon': False,
   'Name': 'FooBar Opportunity AAA',
   'attributes': {'type': 'Opportunity', 'url': '/services/data/foo/bar'}}],
 'totalSize': 1}

In [67]:
@hmemoized
def get(url, params=None, delay=1, **kwargs):
    """Some sort of docstring"""
    return dummy.get(url, params=params, delay=delay)

<function get at 0x7fe5c6448d90>


In [68]:
get(2, {'stuff':1} , spam=True, eggs='yes').json()

Args: (2, {'stuff': 1})
kwArgs: {'eggs': 'yes', 'spam': True}


{'RetrievedDate': '2017-12-13T09:06:46.788012',
 'done': True,
 'records': [{'AccountId': '1337BEEF',
   'Amount': 31415.926,
   'CreatedDate': '2015-07-13T07:30:24.000+0000',
   'IsClosed': True,
   'IsWon': False,
   'Name': 'FooBar Opportunity AAA',
   'attributes': {'type': 'Opportunity', 'url': '/services/data/foo/bar'}}],
 'totalSize': 1}

In [72]:
# get({'stuff': collections})

In [36]:
for obj in collections.__all__:
    cont = getattr(collections, obj)
    try:
        data = cont()
    except:
        pass
    try:
        data = cont('foo', 'bar')
    except:
        pass
    print("{} {}".format(obj, isinstance(data, collections.Hashable)))

deque False
defaultdict False
namedtuple True
UserDict False
UserList False
UserString False
Counter False
OrderedDict False
ChainMap False
Awaitable False
Coroutine False
AsyncIterable False
AsyncIterator False
Hashable False
Iterable False
Iterator False
Generator False
Sized False
Container False
Callable False
Set False
MutableSet False
Mapping False
MutableMapping False
MappingView False
KeysView False
ItemsView False
ValuesView False
Sequence False
MutableSequence False
ByteString False


In [32]:
isinstance({'foo':'bar'}, collections.Hashable)

False